<a href="https://colab.research.google.com/github/shubham7226/005_ShubhamBaldaniya/blob/main/lab06_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
data = pd.read_csv('/content/drive/My Drive/ML/L6/BuyComputer.csv')
data.drop(columns=['User ID',],axis=1,inplace=True)
data.head()

,Age,EstimatedSalary,Purchased
0,19,19000,0
1,35,20000,0
2,26,43000,0
3,27,57000,0
4,19,76000,0


In [9]:
from sklearn.model_selection import train_test_split

y = data.iloc[:,-1].values
X = data.iloc[:,:-1].values
n_samples, n_features = X.shape
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [10]:
from sklearn.preprocessing import StandardScaler
import torch

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [11]:
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [12]:
import torch.nn as nn

class Model(nn.Module):
    def __init__(self, n_input_features):
        super(Model, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

model = Model(n_features)

In [13]:
num_epochs = 140
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [14]:
for epoch in range(num_epochs):
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')


with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'\n Accuracy: {acc.item()*100:.2f}')

epoch: 10, loss = 0.8771
epoch: 20, loss = 0.8558
epoch: 30, loss = 0.8355
epoch: 40, loss = 0.8160
epoch: 50, loss = 0.7974
epoch: 60, loss = 0.7796
epoch: 70, loss = 0.7625
epoch: 80, loss = 0.7463
epoch: 90, loss = 0.7308
epoch: 100, loss = 0.7160
epoch: 110, loss = 0.7019
epoch: 120, loss = 0.6884
epoch: 130, loss = 0.6756
epoch: 140, loss = 0.6634

 Accuracy: 71.25
